In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Step 2: Install necessary libraries
!pip install opencv-python-headless

# Step 3: Import libraries
import cv2
import numpy as np
import os

# Step 4: Define the path to the image on Google Drive
image_path = '/content/drive/MyDrive/image.jpeg'

# Step 5: Load the image
image = cv2.imread(image_path)

# Check if the image is loaded successfully
if image is None:
    raise ValueError("Image not loaded. Please check the image path.")

# Step 6: Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 7: Apply GaussianBlur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Step 8: Apply edge detection
edges = cv2.Canny(blurred, 50, 150)

# Step 9: Find contours based on edges
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Step 10: Find the contour with the largest area, assuming it's the license plate
license_plate_contour = max(contours, key=cv2.contourArea)

# Step 11: Get the bounding box of the license plate contour
x, y, w, h = cv2.boundingRect(license_plate_contour)

# Step 12: Crop the license plate area from the image
license_plate = image[y:y+h, x:x+w]

# Step 13: Convert the cropped license plate to grayscale
license_plate_gray = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)

# Step 14: Apply GaussianBlur to reduce noise
blurred_license_plate = cv2.GaussianBlur(license_plate_gray, (5, 5), 0)

# Step 15: Apply adaptive thresholding
thresh = cv2.adaptiveThreshold(blurred_license_plate, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Step 16: Apply morphological operations to close gaps between contours
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Step 17: Find contours on the thresholded image
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 18: Create a directory to save individual characters
output_dir = '/content/drive/MyDrive/license_plate_characters'
os.makedirs(output_dir, exist_ok=True)

# Step 19: Extract and save individual characters and draw rectangles around them
char_count = 0
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = w / float(h)
    if 0.2 < aspect_ratio < 1.0 and w > 15 and h > 30:  # Filter out non-character contours
        char = license_plate[y:y+h, x:x+w]
        char_path = os.path.join(output_dir, f'char_{char_count}.png')
        cv2.imwrite(char_path, char)
        char_count += 1
        # Draw rectangle around each character
        cv2.rectangle(license_plate, (x, y), (x+w, y+h), (0, 255, 0), 2)

output_image_path = '/content/drive/MyDrive/license_plate_with_boxes.jpg'
cv2.imwrite(output_image_path, image)

print(f'{char_count} characters extracted and saved to {output_dir}')
print(f'Image with boxes saved to {output_image_path}')


Mounted at /content/drive
7 characters extracted and saved to /content/drive/MyDrive/license_plate_characters
Image with boxes saved to /content/drive/MyDrive/license_plate_with_boxes.jpg
